# (1) Hawkess LOB Level-I data and processing 

In [3]:
import json
import pandas as pd
import numpy as np
import os
from datetime import datetime
import asyncio
import nest_asyncio
import ipaddress
from bson import json_util
import pymongo
from datetime import datetime, timedelta
from statistics import mean, stdev

MatchTrade_data = os.listdir("Data/MatchTrades")
MatchTrade_data = [f for f in MatchTrade_data if f != '.ipynb_checkpoints']
#print(MatchTrade_data)

LOB_data = os.listdir("Data/LOB")
LOB_data = [f for f in LOB_data if f != '.ipynb_checkpoints']
#print(LOB_data)

with open("Data/MatchTrades/"+MatchTrade_data[0]) as f:
    MatchTrade_data = pd.read_json(f)
    MatchTrade_data["timestamp"] = pd.to_datetime(MatchTrade_data["timestamp"], unit='ms')
    MatchTrade_data["trade"] = MatchTrade_data[["qty","price","isBuyerMaker","isBestMatch"]].values.tolist()
    MatchTrade_data["item"] = "match trade"
    MatchTrade_data = MatchTrade_data[["timestamp","item","trade"]]
    
with open("Data/LOB/"+LOB_data[0]) as f:
    LOB_data = pd.read_json(f)
    LOB_data["timestamp"] = pd.to_datetime(LOB_data["timestamp"], unit='ms')
    LOB_data["item"] = "lob"
    LOB_data["lob snapshot"] = LOB_data[["bids","asks"]].values.tolist()
    LOB_data = LOB_data[["timestamp","item","lob snapshot"]]

merged = pd.merge(MatchTrade_data, LOB_data, on=["timestamp","item"], how="outer").sort_values("timestamp")
merged = merged.iloc[0:10000]

## (1.2) Making the events file

### (1.2.1) Market Events

- Market buy order
- Market sell order
- Limit buy order
- Limit sell order
- Cancelation buy order
- Cancelation sell order

In [4]:
import pandas as pd
import torch

def make_events(data):

    if not np.issubdtype(type(data), pd.core.frame.DataFrame):
        raise TypeError(f"data is not a pd.DataFrame")
        
    ts_buy_market_orders = []
    ts_sell_market_orders = []
    ts_buy_limit_orders = []
    ts_sell_limit_orders = []
    ts_buy_cancelation_orders = []
    ts_sell_cancelation_orders = []
    
    bids_tensors = None
    asks_tensors = None

    first_time = ((data.iloc[0])["timestamp"]).timestamp()
    
    for index, row in data.iterrows():
        
        if (row["item"] == "lob"):
            
            bids = [float(x) for x in ((row["lob snapshot"])[0])[0]]
            asks = [float(x) for x in ((row["lob snapshot"])[1])[0]]

            # --------------------------------------------- #
            # (1) Limit oders
            # --------------------------------------------- #

            if (asks_tensors != None):

                # Convert each list into a tensor 
                new_bids_tensors = torch.tensor(bids, dtype=torch.float32) 
                new_asks_tensors = torch.tensor(asks, dtype=torch.float32)
    
                new_bid_vol = (new_bids_tensors[1])
                new_ask_vol = (new_asks_tensors[1])

                # ------------------------------------ #
                # (1.1) buy limit order 
                
                if (new_bid_vol > bid_vol):
                    ts_buy_limit_orders.append(row["timestamp"].timestamp() - first_time)
                elif (new_bid_vol < bid_vol):
                    ts_buy_cancelation_orders.append(row["timestamp"].timestamp() - first_time)

                # (1.1) End 
                # ------------------------------------ #

                # ------------------------------------ #
                # (1.2) sell limit order 

                if (new_ask_vol > ask_vol):
                    ts_sell_limit_orders.append(row["timestamp"].timestamp() - first_time)
                elif (new_ask_vol < ask_vol):
                    ts_sell_cancelation_orders.append(row["timestamp"].timestamp() - first_time)

                # (1.2) End 
                # ------------------------------------ #
                
            else:

                bids_tensors = torch.tensor(bids, dtype=torch.float32)
                asks_tensors = torch.tensor(asks, dtype=torch.float32)
                
                bid_vol = (bids_tensors[1])
                ask_vol = (bids_tensors[1])
                
            # --------------------------------------------- #
            # (1) End                                       #
            # --------------------------------------------- #

        
        elif (row["item"] == "match trade"):
            
            if (row["trade"])[3] == True:
                ts_buy_market_orders.append(row["timestamp"].timestamp() - first_time)
            else:
                ts_sell_market_orders.append(row["timestamp"].timestamp() - first_time) 
                
    return [np.array(ts_buy_limit_orders, dtype=float), # type 0: buy limit orders at best bid
            np.array(ts_sell_limit_orders, dtype=float), # type 1: sell limit orders at best ask 
            np.array(ts_buy_market_orders, dtype=float), # type 2: buy market orders
            np.array(ts_sell_market_orders, dtype=float), # type 3: sell market orders
            np.array(ts_buy_cancelation_orders, dtype=float), # type 4: cancel buy limit orders
            np.array(ts_sell_cancelation_orders, dtype=float) # type 5: cancel sell limit orders
           ]

# (2) my Hawkes LOB code


In [6]:
import numpy as np

def simulate_hawkes(mu=0.2, alpha=0.5, beta=1.0, T=50):
    """
    Simulate 1D Hawkes process with exponential kernel using Ogata's thinning.
    
    Parameters:
        mu    - baseline intensity
        alpha - excitation parameter
        beta  - decay rate
        T     - simulation horizon
    """
    t = 0.0
    events = []

    while t < T:
        # Upper bound for intensity
        if len(events) == 0:
            lambda_upper = mu + alpha
            
        else:
            past_terms = np.sum(alpha * np.exp(-beta * (t - np.array(events))))
            lambda_upper = mu + past_terms + alpha  # add alpha as safeguard
            
        # Step 1: propose next event time
        w = np.random.exponential(1.0 / lambda_upper)
        t = t + w
        if t >= T:
            break

        # Step 2: compute true intensity at t
        lambda_t = mu + np.sum(alpha * np.exp(-beta * (t - np.array(events))))

        # Step 3: thinning test
        if np.random.rand() <= lambda_t / lambda_upper:
            events.append(t)

    return np.array(events)

# Example run
np.random.seed(42)
events = simulate_hawkes(mu=0.2, alpha=0.5, beta=1.0, T=50)
print("Simulated event times:", events)

Simulated event times: [ 2.79376951  5.60514394  5.76828948  5.98810429  6.27623925  6.71710193
  6.87924867  7.24070604  7.61529717  8.01060453  9.86394984 12.46564661
 20.49844925 20.53701176 20.82972702 21.74294121 21.93758191 22.84232378
 24.77702134 25.10599801 26.70312826 26.96998188 34.76522599 38.44618794
 38.7283399  43.72421315 48.22644675 48.44101618]


In [12]:
import numpy as np

def simulate_multivariate_hawkes(mu, alpha, beta, T):
    """
    Multivariate Hawkes process simulation with exponential kernels
    using Ogata's thinning.
    
    Parameters:
        mu    : array of baseline intensities (D,)
        alpha : matrix of excitation (D,D)
        beta  : matrix of decay rates (D,D)
        T     : simulation horizon
    Returns:
        events: list of event times per dimension
    """
    
    D = len(mu)
    events = [[] for _ in range(D)]
    t = 0.0
    
    # Convert to numpy for easier operations
    mu = np.array(mu)
    alpha = np.array(alpha)
    beta = np.array(beta)
    
    while t < T:
        # Compute current intensities
        lambda_vals = mu.copy()
        for i in range(D):
            for j in range(D):
                if events[j]:
                    times = np.array(events[j])
                    lambda_vals[i] += np.sum(alpha[i,j] * np.exp(-beta[i,j] * (t - times[times < t])))
        
        # Upper bound for thinning
        M = lambda_vals + np.sum(alpha, axis=1)  # simple safe bound
        M_total = np.sum(M)
        
        if M_total <= 0:
            break
        
        # Step 1: propose next event
        w = np.random.exponential(1.0 / M_total)
        t = t + w
        if t >= T:
            break
        
        # Step 2: choose dimension proportional to M
        probs = M / M_total
        i = np.random.choice(D, p=probs)
        
        # Step 3: accept/reject
        lambda_i = mu[i] + sum(
            np.sum(alpha[i,j] * np.exp(-beta[i,j] * (t - np.array(events[j])))) for j in range(D)
        )
        
        if np.random.rand() <= lambda_i / M[i]:
            events[i].append(t)
    
    return events

# Example: 2-dimensional Hawkes
mu = [0.2, 0.15]
alpha = [[0.3, 0.1],
         [0.05, 0.25]]
beta = [[1.0, 1.0],
        [1.0, 1.0]]

np.random.seed(42)
events = simulate_multivariate_hawkes(mu, alpha, beta, T=50)
print("Event times for each type:", events)


Event times for each type: [[1.3163910892883925, 3.7720680149134687, 3.9596656635791323, 4.373970444245202, 4.474836267285751, 5.220523265716163, 5.743306250408561, 12.305549844154083, 18.194623478457366, 20.561368367055987, 20.664652858078245, 21.282948246444004, 23.453625645832414, 25.186490183406512, 26.009671010015097, 26.25803444252168, 26.70668895272001, 28.20240753792147, 31.293192709819188, 32.84313562279344, 32.8906684166108, 42.96882592675578, 44.00605004236632, 46.57970093298011], [4.820095066712096, 8.317564350229611, 12.099703952598963, 13.077012691553755, 16.29421055119181, 16.299357184146498, 17.21625472199404, 23.351615491079485, 24.628688885182097, 28.7397797542644, 29.55259845362233, 33.93246388110986, 44.880520541051666, 45.19600129359471, 46.80565361644084, 49.07102861366666, 49.09279649419001]]


In [8]:
import numpy as np

def vectorized_multivariate_hawkes_log_likelihood(events, mu, alpha, beta, T):
    """
    Vectorized log-likelihood for multivariate Hawkes process with exponential kernel.

    Parameters:
        events: list of D arrays, each array contains event times for that dimension
        mu: array of length D
        alpha, beta: D x D arrays
        T: observation horizon
    Returns:
        log-likelihood value
    """
    
    D = len(events)
    mu = np.array(mu)
    alpha = np.array(alpha)
    beta = np.array(beta)
    
    ll = 0.0
    
    # Precompute exponentials for integral term
    for i in range(D):
        # Log-likelihood sum: log lambda at each event
        lambdas = np.zeros_like(events[i])
        t_i = events[i]
        for j in range(D):
            t_j = events[j]
            if len(t_j) == 0:
                continue
            # Compute past events contributions using broadcasting
            diff = t_i[:, None] - t_j[None, :]  # shape (len(t_i), len(t_j))
            mask = diff > 0
            contributions = alpha[i, j] * np.exp(-beta[i, j] * diff) * mask
            lambdas += np.sum(contributions, axis=1)
        lambdas += mu[i]
        ll += np.sum(np.log(lambdas))
        
        # Integral term
        integral = mu[i] * T
        for j in range(D):
            t_j = events[j]
            if len(t_j) == 0:
                continue
            integral += np.sum(alpha[i, j] / beta[i, j] * (1 - np.exp(-beta[i, j] * (T - t_j))))
        ll -= integral
        
    return ll

In [9]:
# Example: 2D Hawkes
events = [np.array([0.5, 1.5, 3.0]), np.array([1.0, 2.0, 2.5])]
mu = [0.2, 0.15]
alpha = np.array([[0.3, 0.1],
                  [0.05, 0.25]])
beta = np.array([[1.0, 1.0],
                 [1.0, 1.0]])
T = 4.0

ll = vectorized_multivariate_hawkes_log_likelihood(events, mu, alpha, beta, T)
print("Log-likelihood:", ll)

Log-likelihood: -10.634487051633416


In [10]:
import numpy as np
from scipy.optimize import minimize

# ------------------------------
# MLE wrapper
# ------------------------------

def fit_multivariate_hawkes(events, T, init_params=None):
    """
    Fit a multivariate Hawkes process via MLE.

    events: list of arrays, each array contains event times for one dimension
    T: observation window
    init_params: optional dict {'mu':..., 'alpha':..., 'beta':...}
    """
    D = len(events)
    
    # Flatten parameters for optimization
    if init_params is None:
        mu0 = np.ones(D) * 0.1
        alpha0 = np.ones((D, D)) * 0.1
        beta0 = np.ones((D, D)) * 1.0
    else:
        mu0 = np.array(init_params['mu'])
        alpha0 = np.array(init_params['alpha'])
        beta0 = np.array(init_params['beta'])
    
    def pack_params(mu, alpha, beta):
        return np.concatenate([mu.flatten(), alpha.flatten(), beta.flatten()])
    
    def unpack_params(x):
        idx = 0
        mu = x[idx:idx+D]; idx += D
        alpha = x[idx:idx+D*D].reshape(D, D); idx += D*D
        beta = x[idx:idx+D*D].reshape(D, D)
        return mu, alpha, beta
    
    x0 = pack_params(mu0, alpha0, beta0)
    
    # Negative log-likelihood for minimization
    def neg_ll(x):
        mu, alpha, beta = unpack_params(x)
        if np.any(mu <= 0) or np.any(alpha < 0) or np.any(beta <= 0):
            return np.inf  # enforce positivity
        return -vectorized_multivariate_hawkes_log_likelihood(events, mu, alpha, beta, T)
    
    # Bounds to ensure positivity
    bounds = [(1e-8, None)] * len(x0)
    
    result = minimize(neg_ll, x0, bounds=bounds, method='L-BFGS-B')
    
    mu_hat, alpha_hat, beta_hat = unpack_params(result.x)
    
    return {'mu': mu_hat, 'alpha': alpha_hat, 'beta': beta_hat, 'success': result.success, 'fun': result.fun}


In [ ]:
# 2D Hawkes example
events = make_events(merged)

fit_result = fit_multivariate_hawkes(events, T)
print("Fitted mu:", fit_result['mu'])
print("Fitted alpha:", fit_result['alpha'])
print("Fitted beta:", fit_result['beta'])

/tmp/ipykernel_1350/2008038601.py:35: RuntimeWarning: overflow encountered in exp
  contributions = alpha[i, j] * np.exp(-beta[i, j] * diff) * mask
/tmp/ipykernel_1350/2008038601.py:35: RuntimeWarning: invalid value encountered in multiply
  contributions = alpha[i, j] * np.exp(-beta[i, j] * diff) * mask
/tmp/ipykernel_1350/2008038601.py:46: RuntimeWarning: overflow encountered in exp
  integral += np.sum(alpha[i, j] / beta[i, j] * (1 - np.exp(-beta[i, j] * (T - t_j))))
